# Global BGC metrics

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import xarray as xr

from dask.distributed import Client

import catalog
import util

xr.set_options(keep_attrs=True)

import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import pop_tools
import numpy as np
import pandas as pd

In [ ]:
ds_grid = pop_tools.get_grid('POP_gx1v7')
lons=ds_grid.TLONG
lats=ds_grid.TLAT
area = ds_grid.TAREA
area_m = area/1.e4

### Parameters

In [ ]:
casename = 'g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.4p2z.001'
component = 'pop'
stream = 'h'

cluster_scheduler_address = None

In [ ]:
assert component in ['pop']
assert stream in ['h'] #, 'h.ecosys.nday1']

### Connect to cluster

In [ ]:
if cluster_scheduler_address is None:
    cluster, client = util.get_ClusterClient()
    cluster.scale(12)
else:
    client = Client(cluster_scheduler_address)
client

### Load the data

In [ ]:
%%time

dsets = catalog.to_dataset_dict(
    case=casename,
    component=component,
    stream=stream,
)
dsets.keys()

In [ ]:
ds = dsets[f'{casename}.{component}.{stream}']

### Variable list

In [ ]:
variables =['photoC_TOT_zint',
            'photoC_sp_zint','photoC_diat_zint',
            'photoC_diaz_zint','photoC_cocco_zint',
            'SiO2_PROD','CaCO3_PROD_zint','diaz_Nfix',
            'mesozooC','microzooC','x_graze_microzoo_zint','x_graze_mesozoo_zint',
            'POC_FLUX_100m']

In [ ]:
keep_vars=['z_t','z_t_150m','dz','time_bound','TAREA','TLAT','TLONG'] + variables
ds = ds.drop([v for v in ds.variables if v not in keep_vars])

In [ ]:
dt_wgt = ds.time_bound.diff('d2').squeeze()
dt_wgt /= dt_wgt.sum()
#dt_wgt

In [ ]:
ds = ds.weighted(dt_wgt).mean('time')

### Make Global integrals

##### Carbon-related variables

In [ ]:
cvars = ['photoC_TOT_zint', 'photoC_sp_zint','photoC_diat_zint',
         'photoC_diaz_zint','photoC_cocco_zint', 'POC_FLUX_100m',
         'CaCO3_PROD_zint','x_graze_microzoo_zint','x_graze_mesozoo_zint']

ds_glb = util.global_mean(ds, ds_grid, cvars, normalize=False).compute()

nmols_to_PgCyr = 1e-9 * 12. * 1e-15 * 365. * 86400.

for v in cvars:
    ds_glb[v] = ds_glb[v] * nmols_to_PgCyr        
    ds_glb[v].attrs['units'] = 'Pg C yr$^{-1}$'

##### Percent NPP by diatoms

In [ ]:
ds_glb['diatNPPpercent'] = ds_glb.photoC_diat_zint / ds_glb.photoC_TOT_zint * 100.
ds_glb['diatNPPpercent'].attrs['units'] = '%'

##### Diatom silicification

In [ ]:
tmp = (ds.SiO2_PROD.isel(z_t=slice(0,10)) * 10.).sum(dim='z_t') ## depth integral, units in mmol/m2/s
tmp = tmp * area_m ## mmol/s
tmp = tmp * 86400. * 365 * 0.001 ## mol/yr 
tmp = tmp.sum(dim=('nlon','nlat')) * 1.e-12 #Tmol/yr

ds_glb['SiO2_PROD'] = tmp
ds_glb['SiO2_PROD'].attrs['units'] = 'Tmol Si yr$^{-1}$'

##### Nitrogen fixation

In [ ]:
tmp = (ds['diaz_Nfix'] * 10.).sum(dim='z_t_150m') # unit is mmol/m2/s
tmp = tmp * area_m ## mmol/s
tmp = tmp * 86400. * 365. * 0.001 * 14. * 1e-12 #convert to Tg N / yr

ds_glb['diaz_Nfix'] = tmp.sum(dim=('nlon','nlat'))
ds_glb['diaz_Nfix'].attrs['units'] = 'Tg N yr$^{-1}$'

##### Zooplankton biomass

In [ ]:
tmp = (ds.microzooC * 10.).sum(dim='z_t_150m') #mmol/m2
tmp = (tmp * area_m * 0.001).sum(dim=('nlon','nlat')) ## mol
tmp = tmp.values * 1.e-15 * 12.011 #Pg C
ds_glb['microzooC'] = tmp
ds_glb['microzooC'].attrs['units'] = 'Pg C'

tmp = (ds.mesozooC * 10.).sum(dim='z_t_150m') #mmol/m2
tmp = (tmp * area_m * 0.001).sum(dim=('nlon','nlat')) ## mol
tmp = tmp.values * 1.e-15 * 12.011 #Pg C
ds_glb['mesozooC'] = tmp
ds_glb['mesozooC'].attrs['units'] = 'Pg C'

##### Zooplankton productivity as % NPP

In [ ]:
ds_glb['zooprodNPPpercent'] = (ds_glb['x_graze_mesozoo_zint'] + ds_glb['x_graze_microzoo_zint'])/ds_glb['photoC_TOT_zint'] * 100.
ds_glb['zooprodNPPpercent'].attrs['units'] = '%'

### Make a table of global metrics

In [ ]:
df = pd.DataFrame(columns=['Metric','unit','CESM','Obs','Reference'])

In [ ]:
df.loc[0] = ['NPP',ds_glb.photoC_TOT_zint.attrs['units'],np.round(ds_glb.photoC_TOT_zint.values, 2),'45 to 55','Behrenfeld & Falkowski 1997; Carr et al., 2006']
df.loc[1] = ['POC export 100m',ds_glb.POC_FLUX_100m.attrs['units'],np.round(ds_glb.POC_FLUX_100m.values, 2),'4 to 12','DeVries & Weber, 2017']
df.loc[2] = ['%NPP by diatoms',ds_glb.diatNPPpercent.attrs['units'],np.round(ds_glb.diatNPPpercent.values, 2),'40%','Nelson et al., 1995']
df.loc[3] = ['Silicification',ds_glb.SiO2_PROD.attrs['units'],np.round(ds_glb.SiO2_PROD.values, 2),'100 to 190','Nelson et al., 1995; Holzer et al., 2014']
df.loc[4] = ['Nitrogen fixation',ds_glb.diaz_Nfix.attrs['units'],np.round(ds_glb.diaz_Nfix.values, 2),'125.6 and 222.9','Wang et al., 2019']
df.loc[5] = ['Microzooplankton biomass',ds_glb.microzooC.attrs['units'],np.round(ds_glb.microzooC.values, 2),'0.24','Buitenhuis et al., 2010']
df.loc[6] = ['Mesozooplankton biomass',ds_glb.mesozooC.attrs['units'],np.round(ds_glb.mesozooC.values, 2),'0.16 to 0.19','Buitenhuis et al., 2006; Moriarty & OBrien, 2013']
df.loc[7] = ['Zoo prod % of NPP',ds_glb.zooprodNPPpercent.attrs['units'],np.round(ds_glb.zooprodNPPpercent.values, 2),'~25%','ref?']

In [ ]:
df